In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("C:/Users/Emre/Desktop/customer_behavior_train_useridTR.csv")

In [3]:
df.shape

(455401, 20)

In [14]:
df["satin_aldi"].value_counts(normalize=True)

satin_aldi
0    0.958074
1    0.041926
Name: proportion, dtype: float64

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 455401 entries, 0 to 455400
Data columns (total 20 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   userid                   455401 non-null  object
 1   sepet_ikonuna_tiklama    455401 non-null  int64 
 2   listeden_sepet_ekleme    455401 non-null  int64 
 3   detaydan_sepet_ekleme    455401 non-null  int64 
 4   hesap_sayfasina_tiklama  455401 non-null  int64 
 5   kampanyaya_tiklama       455401 non-null  int64 
 6   favoriye_ekleme          455401 non-null  int64 
 7   beden_secimi             455401 non-null  int64 
 8   mini_sepet_kapatma       455401 non-null  int64 
 9   teslimat_detay_bakma     455401 non-null  int64 
 10  iade_detay_bakma         455401 non-null  int64 
 11  giris_yapma              455401 non-null  int64 
 12  odeme_sayfasi_gorme      455401 non-null  int64 
 13  beden_tablosu_gorme      455401 non-null  int64 
 14  teslimat_bilgisi_gor

In [5]:
df.groupby(["satin_aldi", "beden_tablosu_gorme"]).size()

satin_aldi  beden_tablosu_gorme
0           0                      436152
            1                         156
1           0                       19072
            1                          21
dtype: int64

In [6]:
df = df.drop(columns="beden_tablosu_gorme") # Nadir olduğu için modelden çıkartıldı

In [7]:
kimlik_sutunu = ["userid"]

In [8]:
target = ["satin_aldi"]

In [9]:
df_corr = ["sepet_ikonuna_tiklama", "listeden_sepet_ekleme", "detaydan_sepet_ekleme", "hesap_sayfasina_tiklama", "kampanyaya_tiklama", "favoriye_ekleme", "beden_secimi", "mini_sepet_kapatma", "teslimat_detay_bakma", "iade_detay_bakma", "giris_yapma", "odeme_sayfasi_gorme", "teslimat_bilgisi_gorme", "cihaz_mobil", "cihaz_pc", "cihaz_tablet", "geri_donen_kullanici", "satin_aldi"]

In [10]:
df[df_corr].corr()["satin_aldi"]

sepet_ikonuna_tiklama      0.428334
listeden_sepet_ekleme      0.287666
detaydan_sepet_ekleme      0.414420
hesap_sayfasina_tiklama    0.057279
kampanyaya_tiklama         0.056533
favoriye_ekleme            0.023516
beden_secimi               0.154867
mini_sepet_kapatma         0.140011
teslimat_detay_bakma       0.798720
iade_detay_bakma           0.059484
giris_yapma                0.665556
odeme_sayfasi_gorme        0.708986
teslimat_bilgisi_gorme     0.031461
cihaz_mobil               -0.042907
cihaz_pc                   0.049208
cihaz_tablet               0.016939
geri_donen_kullanici       0.060295
satin_aldi                 1.000000
Name: satin_aldi, dtype: float64

In [11]:
feature_sutunlari = df.drop(["satin_aldi","userid"],axis=1)

In [12]:
feature_sutunlari.head(5)

,sepet_ikonuna_tiklama,listeden_sepet_ekleme,detaydan_sepet_ekleme,hesap_sayfasina_tiklama,kampanyaya_tiklama,favoriye_ekleme,beden_secimi,mini_sepet_kapatma,teslimat_detay_bakma,iade_detay_bakma,giris_yapma,odeme_sayfasi_gorme,teslimat_bilgisi_gorme,cihaz_mobil,cihaz_pc,cihaz_tablet,geri_donen_kullanici
0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
4,0,1,0,0,0,0,1,0,1,0,1,1,0,0,0,1,0


In [ ]:
# Eğitim / test ayrımı ve Logistic Regression, LightGBM, XGBoost modellerinin eğitimi

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from sklearn.linear_model import LogisticRegression

# Gerekirse aşağıdaki satırın yorumunu kaldırarak paketleri yükleyebilirsiniz
# !pip install lightgbm xgboost

from lightgbm import LGBMClassifier
import xgboost as xgb

# Özellik ve hedef ayrımı
X = df.drop(columns=["satin_aldi", "userid"])
y = df["satin_aldi"]

# Eğitim / test ayrımı
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y,
)

print("Egitim seti boyutu:", X_train.shape)
print("Test seti boyutu:", X_test.shape)

# 1) Logistic Regression
logreg = LogisticRegression(max_iter=1000, class_weight="balanced")
logreg.fit(X_train, y_train)

# 0.3 threshold ile tahmin
y_proba_lr = logreg.predict_proba(X_test)[:, 1]
y_pred_lr = (y_proba_lr > 0.3).astype(int)

print("\n=== Logistic Regression (threshold = 0.3) ===")
print("Accuracy:", accuracy_score(y_test, y_pred_lr))
print("ROC-AUC:", roc_auc_score(y_test, y_proba_lr))
print(classification_report(y_test, y_pred_lr))


# 2) LightGBM (sklearn wrapper - LGBMClassifier)
lgbm_clf = LGBMClassifier(
    n_estimators=500,
    learning_rate=0.05,
    num_leaves=31,
    subsample=0.8,
    colsample_bytree=0.9,
    objective="binary",
    subsample_freq=5,
    n_jobs=-1,
)

lgbm_clf.fit(
    X_train,
    y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    eval_metric="auc",
    verbose=False,
)

# 0.3 threshold ile tahmin
y_proba_lgb = lgbm_clf.predict_proba(X_test)[:, 1]
y_pred_lgb = (y_proba_lgb > 0.3).astype(int)

print("\n=== LightGBM (LGBMClassifier, threshold = 0.3) ===")
print("Accuracy:", accuracy_score(y_test, y_pred_lgb))
print("ROC-AUC:", roc_auc_score(y_test, y_proba_lgb))
print(classification_report(y_test, y_pred_lgb))


# 3) XGBoost
xgb_model = xgb.XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="binary:logistic",
    eval_metric="auc",
    tree_method="hist",
    n_jobs=-1,
)

xgb_model.fit(
    X_train,
    y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    verbose=False,
)

# 0.3 threshold ile tahmin
y_proba_xgb = xgb_model.predict_proba(X_test)[:, 1]
y_pred_xgb = (y_proba_xgb > 0.3).astype(int)

print("\n=== XGBoost (threshold = 0.3) ===")
print("Accuracy:", accuracy_score(y_test, y_pred_xgb))
print("ROC-AUC:", roc_auc_score(y_test, y_proba_xgb))
print(classification_report(y_test, y_pred_xgb))



In [ ]:
# LGBM Feature Importance

import matplotlib.pyplot as plt
import seaborn as sns

# Feature importance değerlerini al
feature_importance = lgbm_clf.feature_importances_
feature_names = X_train.columns

# DataFrame oluştur
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importance
}).sort_values('importance', ascending=False)   

print("=== LGBM Feature Importance (Önem Sırasına Göre) ===\n")
print(importance_df.to_string(index=False))

# Görselleştirme
plt.figure(figsize=(10, 8))
sns.barplot(data=importance_df, y='feature', x='importance', palette='viridis')
plt.title('LGBM Feature Importance', fontsize=16, fontweight='bold')
plt.xlabel('Importance', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.tight_layout()
plt.show()



In [ ]:
# Test CSV'yi oku ve yapısını kontrol et
df_test = pd.read_csv("C:/Users/Emre/Desktop/customer_behavior_test_useridTR.csv")

print("Test seti boyutu:", df_test.shape)
print("\nTest seti sütunları:")
print(df_test.columns.tolist())
print("\nİlk 5 satır:")
df_test.head()



In [ ]:
# Test seti için feature'ları hazırla (userid'yi sakla, feature'ları ayır)
test_userid = df_test["userid"]
X_test_final = df_test.drop(columns=["userid"])

# Eğitim setindeki feature sırasını kontrol et (önemli!)
print("Eğitim seti feature'ları:", X_train.columns.tolist())
print("\nTest seti feature'ları:", X_test_final.columns.tolist())

# Feature sırasını eğitim setiyle aynı yap
X_test_final = X_test_final[X_train.columns]

print("\nTest seti feature sayısı:", X_test_final.shape[1])
print("Eğitim seti feature sayısı:", X_train.shape[1])



In [ ]:
# LGBM ile test seti için tahminler

# A) Olasılık skorları (purchase_proba)
test_proba = lgbm_clf.predict_proba(X_test_final)[:, 1]

# B) Sınıf tahminleri (threshold = 0.3 ile)
test_pred = (test_proba > 0.3).astype(int)

print("Test seti tahmin sayısı:", len(test_proba))
print(f"\nOlasılık skorları - Min: {test_proba.min():.4f}, Max: {test_proba.max():.4f}, Ortalama: {test_proba.mean():.4f}")
print(f"\nSınıf tahminleri - 0: {(test_pred == 0).sum()}, 1: {(test_pred == 1).sum()}")



In [ ]:
# A) Çıktı: userid ve purchase_proba (ÖNERİLEN - Skor)
output_proba = pd.DataFrame({
    'userid': test_userid,
    'purchase_proba': test_proba.round(3)  # Noktadan sonra 3 hane
})

# CSV olarak kaydet (float formatı: noktadan sonra 3 hane)
output_proba.to_csv('customer_propensity_scores.csv', index=False, float_format='%.3f')
print("✓ A) Skor çıktısı kaydedildi: customer_propensity_scores.csv")
print(f"  Toplam kayıt: {len(output_proba)}")
print("\nİlk 10 satır:")
print(output_proba.head(10))



In [ ]:
# B) Çıktı: userid ve purchase_pred (OPSİYONEL - Etiket, threshold=0.3)
output_pred = pd.DataFrame({
    'userid': test_userid,
    'purchase_pred': test_pred
})

# CSV olarak kaydet
output_pred.to_csv('customer_propensity_predictions.csv', index=False)
print("✓ B) Etiket çıktısı kaydedildi: customer_propensity_predictions.csv")
print(f"  Toplam kayıt: {len(output_pred)}")
print(f"  Threshold: 0.3")
print(f"  Tahmin edilen satın alan (1): {(test_pred == 1).sum()}")
print(f"  Tahmin edilen satın almayan (0): {(test_pred == 0).sum()}")
print("\nİlk 10 satır:")
print(output_pred.head(10))



In [ ]:
# Yan rapor: Top %5 / %10 ve niyet segmentleri

# Çalışmak için skor DataFrame'ini kullanıyoruz: output_proba (userid, purchase_proba)
report_df = output_proba.copy()
# purchase_proba zaten 3 haneli formatlanmış durumda

# Top %5 ve %10 eşiklerini bul
p95 = report_df['purchase_proba'].quantile(0.95)
p90 = report_df['purchase_proba'].quantile(0.90)

report_df['top_5_flag'] = (report_df['purchase_proba'] >= p95).astype(int)
report_df['top_10_flag'] = (report_df['purchase_proba'] >= p90).astype(int)

# Niyet segmenti:
# purchase_proba >= 0.7  -> 'High'
# 0.4 <= purchase_proba < 0.7 -> 'Medium'
# purchase_proba < 0.4  -> 'Low'

def segment_func(p):
    if p >= 0.7:
        return 'High'
    elif p >= 0.4:
        return 'Medium'
    else:
        return 'Low'

report_df['intent_segment'] = report_df['purchase_proba'].apply(segment_func)

# Sıralı hale getir (yüksek skordan düşüğe)
report_df = report_df.sort_values('purchase_proba', ascending=False).reset_index(drop=True)

# Raporu kaydet (purchase_proba için float formatı: noktadan sonra 3 hane)
report_df.to_csv('customer_propensity_side_report.csv', index=False, float_format='%.3f')

print('✓ Yan rapor kaydedildi: customer_propensity_side_report.csv')
print(f"Toplam kayıt: {len(report_df)}")
print('\nİlk 20 kayıt (yüksekten düşüğe sıralı):')
report_df.head(20)
